## Generate unfocused radargram using `unfoc`

This notebook uses the UTIG `unfoc` library to generate and plot unfocused radargrams with
time information from the GPS stream.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import gzip
import pandas as pd
from pathlib import Path
from scipy.interpolate import interp1d

import unfoc

from utig_radar_loading import file_util, stream_util, geo_util

### Find a file to process

In [3]:
use_cache = True
cache_dir = "outputs/file_index.csv"
base_path = "/kucresis/scratch/data/UTIG"

df_files = file_util.load_file_index_df(base_path, cache_dir, read_cache=use_cache)
df_artifacts = file_util.create_artifacts_df(df_files)

Reading from cache file outputs/file_index.csv


/kucresis/scratch/tteisberg_sta/scripts/python/utig_radar_loading/src/utig_radar_loading/file_util.py:21: DtypeWarning: Columns (14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_files = pd.read_csv(cache_path, index_col=0)


In [4]:
#r_prj, r_set, r_trn = 'PEL', 'JKB2u', 'Y20b'
#r_prj, r_set, r_trn = 'ICP10', 'JKB2u', 'F01T02a'
#r_prj, r_set, r_trn = 'PEL', 'JKB2u', 'X48a'
r_prj, r_set, r_trn = 'VFT', 'JKB2r', 'Y74a'

df_filt = df_artifacts[
    (df_artifacts['prj'] == r_prj) &
    (df_artifacts['set'] == r_set) &
    (df_artifacts['trn'] == r_trn)
]
df_filt_rad = df_filt[(df_filt['stream'].isin(['RADnh5'])) & (df_filt['file_name'] == 'bxds')]
df_filt_gps = df_filt[df_filt['stream'].isin(['GPSnc1', 'GPStp2'])]

df_filt

,dataset,processing_level,processing_type,prj,set,trn,stream,file_name,full_path,artifact
2373,UTIG1,orig,xlob,VFT,JKB2r,Y74a,RADnh5,xds.gz,/kucresis/scratch/data/UTIG/UTIG1/orig/xlob/VF...,"(orig, xlob, RADnh5)"
19881,UTIG1,targ,pcor,VFT,JKB2r,Y74a,AQNnr1,xds.gz,/kucresis/scratch/data/UTIG/UTIG1/targ/pcor/VF...,"(targ, pcor, AQNnr1)"
19882,UTIG1,targ,pcor,VFT,JKB2r,Y74a,AVNcp1,xds.gz,/kucresis/scratch/data/UTIG/UTIG1/targ/pcor/VF...,"(targ, pcor, AVNcp1)"
19883,UTIG1,targ,pcor,VFT,JKB2r,Y74a,AVNcp2,xds.gz,/kucresis/scratch/data/UTIG/UTIG1/targ/pcor/VF...,"(targ, pcor, AVNcp2)"
19884,UTIG1,targ,pcor,VFT,JKB2r,Y74a,AVNnt2,xds.gz,/kucresis/scratch/data/UTIG/UTIG1/targ/pcor/VF...,"(targ, pcor, AVNnt2)"
19885,UTIG1,targ,pcor,VFT,JKB2r,Y74a,GPSap3,xds.gz,/kucresis/scratch/data/UTIG/UTIG1/targ/pcor/VF...,"(targ, pcor, GPSap3)"
19886,UTIG1,targ,pcor,VFT,JKB2r,Y74a,GPSkc1,xds.gz,/kucresis/scratch/data/UTIG/UTIG1/targ/pcor/VF...,"(targ, pcor, GPSkc1)"
19887,UTIG1,targ,pcor,VFT,JKB2r,Y74a,GPSnc1,xds.gz,/kucresis/scratch/data/UTIG/UTIG1/targ/pcor/VF...,"(targ, pcor, GPSnc1)"
19888,UTIG1,targ,pcor,VFT,JKB2r,Y74a,LASrz1,xds.gz,/kucresis/scratch/data/UTIG/UTIG1/targ/pcor/VF...,"(targ, pcor, LASrz1)"
19889,UTIG1,targ,pcor,VFT,JKB2r,Y74a,MAGgm2,xds.gz,/kucresis/scratch/data/UTIG/UTIG1/targ/pcor/VF...,"(targ, pcor, MAGgm2)"


In [5]:
if len(df_filt_rad) > 1:
    raise Exception('More filtering needed to get a single file')

artifact = df_filt_rad.iloc[0]
data_dir = Path(artifact['full_path']).parent
bxds_file_path = data_dir / "bxds"
if bxds_file_path.exists():
    print(f"Found bxds file to load at: {bxds_file_path}")
    stream_type = unfoc.read.get_radar_stream(bxds_file_path)
    print(f"Detected stream type: {stream_type}")
    radar_instrument, channels = unfoc.read.get_radar_type(bxds_file_path)
    print(f"Detected radar instrument: {radar_instrument}, channels: {channels}")

Found bxds file to load at: /kucresis/scratch/data/UTIG/UTIG1/orig/xlob/VFT/JKB2r/Y74a/RADnh5/bxds
Detected stream type: RADnh5
Detected radar instrument: MARFA, channels: [1, 2, 3, 4]


### Manually pulse compress a single range line

In [ ]:
n_sum_coherent = 10
start_trace_idx = 0

# Load n_sum_coherent traces from a single channel
channel = 1  # Use channel 1

# Use RadBxds to read traces from the bxds file
bxds_reader = unfoc.RadBxds(str(bxds_file_path), channel=channel)
print(f"Opened bxds file for channel {channel}")
print(f"Total traces available: {len(bxds_reader)}")
print(f"Trace dimensions: {bxds_reader.shape}")

# Load the traces for coherent stacking
traces_complex = bxds_reader[start_trace_idx:start_trace_idx + n_sum_coherent]
print(f"Loaded {traces_complex.shape[0]} traces with {traces_complex.shape[1]} samples each")

# Coherently stack these traces into a single trace
stacked_trace = np.mean(traces_complex, axis=0)
print(f"Coherently stacked trace shape: {stacked_trace.shape}")

# Load the reference waveform to pulse compress with
# The reference chirp is obtained from the get_ref_chirp function
ref_chirp_fft = unfoc.get_ref_chirp(bandpass=False, trunc_sweep_length=traces_complex.shape[1])
print(f"Reference chirp FFT shape: {ref_chirp_fft.shape}")

# Pulse compress the coherently stacked trace using denoise_and_dechirp

# compressed_trace = unfoc.denoise_and_dechirp(
#     trace=stacked_trace.copy(),
#     ref_chirp=ref_chirp_fft,
#     blanking=50,
#     output_samples=traces_complex.shape[1],
#     do_cinterp=False
# )

DFT = np.fft.fft(stacked_trace)
Product = np.multiply(ref_chirp_fft, DFT)
compressed_trace = np.fft.ifft(Product)

print(f"Compressed trace shape: {compressed_trace.shape}")

# Plot the result
fig, axes = plt.subplots(4, 1, figsize=(12, 8))

# Plot magnitude in dB
magnitude_db = 20 * np.log10(np.abs(compressed_trace) + 1e-10)
axes[0].plot(magnitude_db)
axes[0].set_xlabel('Sample Number')
axes[0].set_ylabel('Power (dB)')
axes[0].set_title('Pulse Compressed Trace - Magnitude')
axes[0].grid(True, alpha=0.3)

# Plot phase
phase = np.angle(compressed_trace)
axes[1].plot(phase)
axes[1].set_xlabel('Sample Number')
axes[1].set_ylabel('Phase (radians)')
axes[1].set_title('Pulse Compressed Trace - Phase')
axes[1].grid(True, alpha=0.3)

# Plot refernece chirp
axes[2].plot(np.real(ref_chirp_fft), label='Real Part')
axes[2].plot(np.imag(ref_chirp_fft), label='Imaginary Part')
axes[2].set_xlabel('Sample Number')
axes[2].set_ylabel('Amplitude')
axes[2].set_title('Reference Chirp (FFT)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

# Plot the original stacked trace for comparison
axes[3].plot(np.real(stacked_trace), label='Real Part')
axes[3].plot(np.imag(stacked_trace), label='Imaginary Part')
axes[3].set_xlabel('Sample Number')
axes[3].set_ylabel('Amplitude')
axes[3].set_title('Original Stacked Trace')
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Close the reader when done
bxds_reader.close()

### Pulse compress entire file

In [ ]:
unfoc.unfoc(
    infile=str(bxds_file_path),
    outdir='outputs/unfoc_scratch/',
    stackdepth=10, incodepth=1,
    channels='LoResInco1,LoResInco2',
    processes=2,
    output_samples=3200,
    blanking=50,
    bandpass=False
    )


### Load and align GPS data with radar data

In [ ]:
ct_df = stream_util.load_ct_file(bxds_file_path)
gps_path = df_filt_gps.iloc[0]['full_path']
print(f"Found GPS stream file: {gps_path}")
gps_df = stream_util.load_gzipped_stream_file(gps_path, parse=True, parse_kwargs={'use_ct': True})

# Nearest neighbor interpolation of GPS data to CT data using tim as common index

# Create nearest neighbor interpolators for each GPS field
lat_interp = interp1d(gps_df['tim'].values, gps_df['LAT'].values, kind='nearest', fill_value='extrapolate')
lon_interp = interp1d(gps_df['tim'].values, gps_df['LON'].values, kind='nearest', fill_value='extrapolate')
timestamp_interp = interp1d(gps_df['tim'].values, gps_df['TIMESTAMP'].values, kind='nearest', fill_value='extrapolate')

# Interpolate GPS data to CT tim values
ct_df['LAT'] = lat_interp(ct_df['tim'].values)
ct_df['LON'] = lon_interp(ct_df['tim'].values)
ct_df['TIMESTAMP'] = timestamp_interp(ct_df['tim'].values)
ct_df['TIMESTAMP'] = pd.to_datetime(ct_df['TIMESTAMP'], unit='ns')
ct_df = ct_df.set_index('seq')

print(f"Interpolated {len(ct_df)} CT records with GPS data")
ct_df.head()

### Plot radargram

In [ ]:
# Load and plot both MagLoResInco1 and MagLoResInco2 with time axis
def load_radar_data(channel_name):
    """Load radar data for a given channel."""
    # Read trace numbers
    trace_file = Path(f'outputs/unfoc_scratch/TraceNumbers{channel_name[-1]}')
    with open(trace_file, 'r') as f:
        trace_numbers = [int(line.strip()) for line in f]
    
    # Read metadata
    meta_file = Path(f'outputs/unfoc_scratch/{channel_name}.meta')
    metadata = {}
    with open(meta_file, 'r') as f:
        for line in f:
            if line.startswith('#') and '=' in line:
                key, value = line[1:].split('=', 1)
                metadata[key.strip()] = value.strip().strip('"')
    
    # Read binary data
    data_file = Path(f'outputs/unfoc_scratch/{channel_name}')
    file_size = data_file.stat().st_size
    num_traces = len(trace_numbers)
    num_samples = file_size // 4 // num_traces
    
    with open(data_file, 'rb') as f:
        binary_data = f.read()
    
    # Convert data
    magscale = int(metadata.get('Scale', '20000'))
    data_int = np.frombuffer(binary_data, dtype='>i4').reshape(num_traces, num_samples)
    data = 10.0 ** (data_int.astype(np.float32) / magscale)
    data_db = 20 * np.log10(data + 1e-10)
    
    # Get timestamps
    trace_timestamps = np.array(ct_df.loc[trace_numbers]['TIMESTAMP'])
    
    return data_db, trace_timestamps, num_traces, num_samples

# Load both channels
channels = ['MagLoResInco1', 'MagLoResInco2']
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

for idx, (channel, ax) in enumerate(zip(channels, axes)):
    data_db, trace_timestamps, num_traces, num_samples = load_radar_data(channel)
    
    # Plot radargram
    extent = [trace_timestamps[0], trace_timestamps[-1], num_samples, 0]
    im = ax.imshow(data_db.T, aspect='auto', cmap='gray',
                   extent=extent,
                   vmin=np.percentile(data_db, 5),
                   vmax=np.percentile(data_db, 99.9))
    
    # Labels and title
    ax.set_ylabel('Sample Number')
    ax.set_title(f'{channel} | {r_prj} {r_set} {r_trn}')
    
    # Colorbar
    plt.colorbar(im, ax=ax, label='Power (dB)')
    
    # Add trace numbers on top axis for first subplot
    if idx == 0:
        ax2 = ax.twiny()
        ax2.set_xlim(0, num_traces)
        ax2.set_xlabel('Trace Number')

# Set x-label only for bottom subplot
axes[-1].set_xlabel('GPS Timestamp')

plt.tight_layout()
plt.show()

fig.savefig(f"outputs/unfoc_radargrams/{r_prj}_{r_set}_{r_trn}.png")

# Print statistics
print(f"Radargram dimensions: {num_traces} traces x {num_samples} samples")
print(f"Time span: {(trace_timestamps[-1] - trace_timestamps[0]).astype('timedelta64[m]')}")

In [ ]:
# Plot the first trace with sample number on X axis and Power (dB) on Y axis
first_trace_db = data_db[0, :]  # Get the first trace
sample_numbers = np.arange(num_samples)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(sample_numbers, first_trace_db, 'b-', linewidth=0.5)
ax.set_xlabel('Sample Number')
ax.set_ylabel('Power (dB)')
ax.set_title(f'First Trace Power Profile | {r_prj} {r_set} {r_trn}')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print statistics for the first trace
print(f"First trace statistics:")
print(f"  Min power: {first_trace_db.min():.2f} dB")
print(f"  Max power: {first_trace_db.max():.2f} dB")
print(f"  Mean power: {first_trace_db.mean():.2f} dB")
print(f"  Number of samples: {num_samples}")